<a href="https://colab.research.google.com/github/seacabezasma/IIS_2020_01_Taller_01/blob/develop-seacabezasma/Taller_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1
### [Introducción a los sistemas inteligentes 2020-1](https://fagonzalezo.github.io/iis-2020-1/)

Integrantes del grupo (máximo 3):

* Nombre_1 ID_1
* Nombre_2 ID_2
* Nombre_3 ID_3

___________

**Fecha límite de entrega**: _Lunes 13 de Abril_ antes de la medianoche 
Envios fuera de este plazo no se tendrán en cuenta.


Cerciórese de reiniciar y correr el notebook en su totalidad antes de enviarlo. Verifique que todas las salidas se muestran de manera correcta.

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/OapI3DaGbxdtWoUkpLiR) antes de la hora límite. El archivo debe nombrarse como  `isi-taller1-unalusername1-unalusername2-unalusername3.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).

___________

## Cuadrado de Rubik (o Cubo de Rubik 2D)

El *Cuadrado de Rubik* es un rompecabezas inspirado en el famoso cubo de Rubik. Consiste de un arreglo de 16 fichas organizadas en una matriz de $4\times 4$ como se ilustra en la siguiente figura:

<img src="https://raw.githubusercontent.com/fagonzalezo/iis-2018-2/master/rubik2D.png" 
alt="Cuadrado de Rubik " width="240" height="180" border="10" />

Los colores son ilustrativos, lo importante es el número en cada una de las fichas. Se pueden hacer 10 movimientos diferentes correspondientes a rotar las 4 fichas alrededor de cada uno de los puntos A, B, C, D y E en el sentido de las manecillas del reloj o en el sentido opuesto.

Su objetivo es modelar el *Cuadrado de Rubik* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Cuadrado de Rubik

Un Cuadrado de Rubik debe representarse como una lista con valores enteros que representan cada una de las fichas.

Por ejemplo un Cuadrado de Rubik resuelto debe verse así:

```python
[ 1,  2,  3,  4,
  5,  6,  7,  8,
  9, 10, 11, 12,
 13, 14, 15, 16]
```

#### Definición de acciones

La siguiente lista define las posibles acciones que se pueden ejecutar:

In [0]:
'''
These values MUST not be changed.
They represent the movements of the Rubik's Square.
'''
actions = ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

Cada acción indica la posición y el sentido. Por ejemplo, `'C-'` rota la posición C en el sentido opuesto de las manecillas del reloj. Si aplicamos esta acción al estado solución se obtiene el estado:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 10, 12,
 13, 14, 15, 16]
```

Si sobre este estado, aplicamos la acción `'E+'` obtenemos:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 15, 10,
 13, 14, 16, 12]
```

#### Clase Rubik2D_problem

In [0]:
# This is the Problem class from AIMA, you don't have to modify it

%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

    
class rubik2d_problem(Problem):    
    
    def __init__(self, initial):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15]
        [1, 9, 4, 8, 5, 6, 3, 2, 15, 10, 11, 12, 13, 14, 7, 16]
        [2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12]
        '''
        self.expanded = 0
        self.initial, self.goal = initial, sorted(initial)
        
    
    def actions(self, state):
        """Return a list of actions that can be executed in the given
        state."""
        return ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

    def result(self, state, action):
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        # Your code here
        if action == "A+":
          rotation = [state [0], state[1], state[4], state[5]]
          state[0] = rotation[2]
          state[1] = rotation[0]
          state[4] = rotation[3]
          state[5] = rotation[1]

        if action == "A-":
          rotation = [state [0], state[1], state[4], state[5]]
          state[0] = rotation[1]
          state[1] = rotation[3]
          state[4] = rotation[0]
          state[5] = rotation[2]

        if action == "B+":
          rotation = [state [2], state[3], state[6], state[7]]
          state[2] = rotation[2]
          state[3] = rotation[0]
          state[6] = rotation[3]
          state[7] = rotation[1]

        if action == "B-":
          rotation = [state [2], state[3], state[6], state[7]]
          state[2] = rotation[1]
          state[3] = rotation[3]
          state[6] = rotation[0]
          state[7] = rotation[2]

        if action == "C+":
          rotation = [state [5], state[6], state[9], state[10]]
          state[5] = rotation[2]
          state[6] = rotation[0]
          state[9] = rotation[3]
          state[10] = rotation[1]

        if action == "C-":
          rotation = [state [5], state[6], state[9], state[10]]
          state[5] = rotation[1]
          state[6] = rotation[3]
          state[9] = rotation[0]
          state[10] = rotation[2]

        if action == "D+":
          rotation = [state [8], state[9], state[12], state[13]]
          state[8] = rotation[2]
          state[9] = rotation[0]
          state[12] = rotation[3]
          state[13] = rotation[1]

        if action == "D-":
          rotation = [state [8], state[9], state[12], state[13]]
          state[8] = rotation[1]
          state[9] = rotation[3]
          state[12] = rotation[0]
          state[13] = rotation[2]

        if action == "E+":
          rotation = [state [10], state[11], state[14], state[15]]
          state[10] = rotation[2]
          state[11] = rotation[0]
          state[14] = rotation[3]
          state[15] = rotation[1]

        if action == "E-":
          rotation = [state [10], state[11], state[14], state[15]]
          state[10] = rotation[1]
          state[11] = rotation[3]
          state[14] = rotation[0]
          state[15] = rotation[2]

        return state[:]

    def is_goal(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        # Your code here
        return state == self.goal


    def action_cost(self, s, a, s1): 
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        # Your code here
        return 0



### 2. Evalue su código con diferentes estrategias de búsqueda



In [40]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {}
    reached[tuple(problem.initial)] = node
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if tuple(s) not in reached or child.path_cost < reached[tuple(s)].path_cost:
                reached[tuple(s)] = child
                frontier.add(child)
    return failure


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {tuple(problem.initial)}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if tuple(s) not in reached:
                reached.add(tuple(s))
                frontier.appendleft(child)
    return failure

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

def bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)
    #return breadth_first_search(problem)

def iterativeDeepeningSearch(problem):
    ### your code here ###
    return iterative_deepening_search(problem)

problem = rubik2d_problem( [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15])

#actions = path_states(bfs(problem))
#for act in actions:
#  print(act)

actions = path_states(iterativeDeepeningSearch(problem))
for act in actions:
  print(act)



[1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15]
[1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 8, 12, 14, 10, 15, 16]
[1, 2, 7, 3, 5, 11, 9, 4, 13, 8, 6, 12, 14, 10, 15, 16]
[1, 2, 7, 3, 5, 8, 11, 4, 13, 6, 9, 12, 14, 10, 15, 16]
[1, 2, 7, 3, 5, 6, 8, 4, 13, 9, 11, 12, 14, 10, 15, 16]
[1, 2, 7, 3, 5, 6, 8, 4, 14, 13, 11, 12, 10, 9, 15, 16]
[1, 2, 7, 3, 5, 6, 8, 4, 10, 14, 11, 12, 9, 13, 15, 16]
[1, 2, 7, 3, 5, 6, 8, 4, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 8, 7, 5, 6, 4, 3, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 4, 8, 5, 6, 3, 7, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


Evaluelo para ver cuál es la máxima profundidad que se puede alcanzar en un tiempo razonable con cada estrategia de búsqueda. Reporte los resultados.

### 3. Implemente diferente heurísticas para el problema

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de [Russell10].

In [0]:
def nullHeuristic(state):
    return 0

def myHeuristic(state):
    ### your code here ###
    return 0

def aStarSearch(problem, h=nullHeuristic):
    ### your code here ###
    return []



In [0]:
problem = rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print(iterativeDeepeningSearch(problem))
print(aStarSearch(problem, myHeuristic))

[]
[]
